In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

AZURE API

In [2]:
from openai import AzureOpenAI
# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    azure_endpoint="https://team5-chatgpt-4-api.openai.azure.com/",
    api_version = "2023-05-15",  # Use the latest available version
    api_key = "9423fcf02a494b5cbe440c6971903ba7",
)

In [3]:
#read the dataset from the jsonl file
df = pd.read_json('DATASETS\human-eval-modified.jsonl', lines=True)

df.head()




,task_id,prompt,entry_point,canonical_solution,test
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,has_close_elements,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,separate_paren_groups,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,truncate_number,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,below_zero,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean_absolute_deviation,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."


Generating the test cases as a list

In [71]:
# chat completion
responses = pd.DataFrame(columns=['prompt', 'entry_point', 'response'])

prompts = df['prompt'].tolist()
entry_points = df['entry_point'].tolist()

for i in range(20):
    problem_statement = prompts[i]
    entry_point = entry_points[i]
        
    response = client.chat.completions.create(
        model="gpt4-api",  # The name you used when deploying the model
        messages=[
            {"role": "system", "content": "You are a helpful Python programming assistant. Your objective is to create test inputs for the given problem."},
            {"role": "user", "content": f"""For the given problem statement, create test inputs. The format of the input is a list of inputs.
                                        The problem statement is: {problem_statement}.
                                        The entry point is: {entry_point},
                                        The input list is 10 inputs representing 10 test cases. Cover basic test cases and edge cases.
                                        Return only the function calls with each of the test cases, each in a separate line, without any other text."""},
        ]
    )

    #save the response to a variable
    response_text = response.choices[0].message.content
    #create a temp dataframe to store the responses
    temp_df = pd.DataFrame([[problem_statement, entry_point, response_text]], columns=['prompt', 'entry_point', 'response'])
    #concatenate the temp dataframe to the responses dataframe
    responses = pd.concat([responses, temp_df], ignore_index=True)
    

In [72]:
for row in responses.iterrows():
    print("Problem Statement: ", row[1]['prompt'])
    print("Entry Point: ", row[1]['entry_point'])
    print("Response: ", row[1]['response'])
    print("\n")
    print(50*"-")   

Problem Statement:  from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Entry Point:  has_close_elements
Response:  [
    has_close_elements([1.0, 2.0, 3.0], 0.5),
    has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3),
    has_close_elements([], 1.0),
    has_close_elements([3.5], 0.7),
    has_close_elements([1.005, 1.004], 0.001),
    has_close_elements([3.141592, 3.141593], 0.000001),
    has_close_elements([-1.0, -2.0, -3.0], 1.1),
    has_close_elements([1000000.0, 1000001.0, 2000000.0], 1.5),
    has_close_elements([1.01, 1.02, 1.0301, 1.0402], 0.01),
    has_close_elements([2.5, 5.0, 7.5, 10.0, 12.5], 2.6)
]


--------------------------------------------------
Problem Statement:  f

Parsing the test cases as a list

In [73]:
for row in responses.iterrows():
    print("Response: ", row[1]['response'])
    print("\n")
    print(50*"-")   

    print("Problem Statement: ", row[1]['prompt'])

    #parse the response to get the list of inputs
    inputs = row[1]['response'].split("\n")
    #remove any input that doesn't have the entry point
    inputs = [input for input in inputs if row[1]['entry_point'] in input]
    for input in inputs:
        print("Input: ", input)
    print("\n")
    print(50*"-")


Response:  [
    has_close_elements([1.0, 2.0, 3.0], 0.5),
    has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3),
    has_close_elements([], 1.0),
    has_close_elements([3.5], 0.7),
    has_close_elements([1.005, 1.004], 0.001),
    has_close_elements([3.141592, 3.141593], 0.000001),
    has_close_elements([-1.0, -2.0, -3.0], 1.1),
    has_close_elements([1000000.0, 1000001.0, 2000000.0], 1.5),
    has_close_elements([1.01, 1.02, 1.0301, 1.0402], 0.01),
    has_close_elements([2.5, 5.0, 7.5, 10.0, 12.5], 2.6)
]


--------------------------------------------------
Problem Statement:  from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Input:      has_close_elements([1.0, 2.0, 3.0], 0.5),
